# Importar bibliotecas

In [0]:
import pandas as pd
import numpy as np

# Leitura do arquivo fa084_tch_mult.csv

In [0]:
# Leitura do csv
df = pd.read_csv('fa084_tch_mult.csv')

* Excluir o atributo 'espac'
* Excluir os registros com estágio de corte 7
* Considerar as três variedades mais frequentes e excluir as demais

In [0]:
# Excluir 'espac'
df = df.drop('espac',axis=1)

# Excluir os registros com estágio de corte 7
df = df.drop(df[df.estagio == 7].index)

# Considerar somente as três variedades mais frequentes e excluir as demais
row_retain = df.variedade.value_counts()[:3].index.tolist()
df = df[df.variedade.isin(row_retain)]

In [0]:
# Check dimensões do arquivo
df.shape

(8565, 13)

## OneHotEncode (Atributos 'variedade' e 'solo')

In [0]:
df = pd.get_dummies(df)

# Construção de Modelos

## Importar pacotes do Scikit-Learn necessários para construção e avaliação dos **modelos**

In [0]:
# Conjunto Treino/Teste
from sklearn.model_selection import train_test_split

# Calcular MAE e MSE/RMSE
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error

# SVR - SVM for Regression 
from sklearn.svm import SVR

## Dividir Conjunto de dados em Treino/Test na proporção 70/30 (random_state = 2020)

In [0]:
# Separar atributos preditores(X) do atributo meta (y)
y = df.tch
X = df.drop(['tch'],axis=1)

# Criar X_train, X_test, y_train, y_test com as proporções 70/30
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 2020, test_size=0.3)

## Construir modelo de regressão com SVM (SVR) - SEM OTIMIZAR HIPER PARÂMETROS
* kernel: rbf

In [0]:
# Definir regressor
regressor = SVR()

# Fazer ajuste
modSVM = regressor.fit(X_train,y_train)

# Fazer predição
y_pred = modSVM.predict(X_test)

# Calcular RMSE
rmse_modSVM = np.sqrt(mean_squared_error(y_test,y_pred))

# Calcular MAE
mae_modSVM = mean_absolute_error(y_test,y_pred)

In [0]:
print('mae =',mae_modSVM, 'rmse =', rmse_modSVM)

mae = 16.00963738792142 rmse = 19.967066186428195


In [0]:
# Só para comparação com os valores obtidos anteriormente com outros modelos
# TODOS SEM OTIMIZAÇÃO DE HIPER PARÂMETROS

# Ordem crescente MAE/RMSE
# RandomForest:      mae =  6.3641857548642, rmse =  9.80230887116812
# Árvore de Decisão: mae =  6.8783055123217, rmse = 13.28448673338847
# XGBoost:           mae = 11.0125962597822, rmse = 14.01084204054599
# GBoost:            mae = 11.0380906475385, rmse = 14.09045428593204
# AdaBoost:          mae = 13.4491333634774, rmse = 16.34247797291942
# Linear Regression: mae = 13.7228268860628, rmse = 17.35946434934895

## OTIMIZAR HIPER PARÂMETROS (GridSearchCV)
* kernel: rbf

### Importar pacotes do Scikit-Learn necessários para construção otimizar hiper parâmetros

In [0]:
# Importar GridSearch
from sklearn.model_selection import GridSearchCV 

In [0]:
# Definir o regressor (Caso_Grid_1)
reg_grid_1 = SVR()
param_grid = [{'gamma':[0.1,1,10],
               'C':[0.1,1,10,100]}]

SVMGSearch_1 = GridSearchCV(reg_grid_1,param_grid=param_grid,cv=5)
SVMGSearch_1.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.1, 1, 10, 100], 'gamma': [0.1, 1, 10]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [0]:
# Identificar o melhores valores para os parâmetros
otim_param_grid_1 = SVMGSearch_1.best_params_
otim_param_grid_1

{'C': 100, 'gamma': 0.1}

In [0]:
# Face aos resultados obtidos, vamos alterar a faixa de valores de gamma e C.

# Considerando que o tempo de execução é MUITO GRANDE, vamos permitir a paralelização
# E medir o tempo de execução (Caso_Grid_2)
%%time

# Definir o regressor
reg_grid_2 = SVR()
param_grid = [{'gamma':[0.01,0.1,1],
               'C':[100,500,1000]}]

SVMGSearch_2 = GridSearchCV(reg_grid_2,param_grid=param_grid,cv=5,n_jobs=3)
SVMGSearch_2.fit(X_train, y_train)

# Incluir n_jobs, para processamento paralelizado (lembrar de ativar GPU)



CPU times: user 7.18 s, sys: 82 ms, total: 7.26 s
Wall time: 2min 18s


In [0]:
# Identificar o melhores valores para os parâmetros
otim_param_grid_2 = SVMGSearch_2.best_params_
otim_param_grid_2

{'C': 500, 'gamma': 0.01}

## OTIMIZAR HIPER PARÂMETROS (RandomizedSearchCV)
* kernel: rbf

In [0]:
# Importar RandomizedSearch
from sklearn.model_selection import RandomizedSearchCV

In [0]:
%%time

# Definir o regressor (Caso_Rnd_1)
reg_rnd_1 = SVR(kernel='rbf')
p_range = [{'gamma':np.arange(0.01,1000,0.01),
               'C':np.arange(1e-3,1e3,1e-3)}]


SVMRndSearch_1 = RandomizedSearchCV(reg_rnd_1,
                                    param_distributions = p_range,
                                    n_iter=9,
                                    n_jobs=3,
                                    random_state=2020)

SVMRndSearch_1.fit(X_train,y_train)

CPU times: user 4.75 s, sys: 82 ms, total: 4.83 s
Wall time: 1min 37s


In [0]:
# Identificar o melhores valores para os parâmetros
otim_param_rnd_1 = SVMRndSearch_1.best_params_
otim_param_rnd_1

{'C': 138.294, 'gamma': 13.93}

## Executar o SVM com os parâmetros ótimos para avaliar MAE e RMSE no conjunto de teste

In [0]:
# Definir regressor com kernel rbf e parâmetros ótimos
reg_otimo = SVR(kernel='rbf',
                gamma = 13.93,
                C = 138.294)

# Fazer o ajuste
modSVM_otimo = reg_otimo.fit(X_train,y_train)

# Fazer predição
y_pred = modSVM_otimo.predict(X_test)

# Calcular RMSE
rmse_modSVM_otimo = np.sqrt(mean_squared_error(y_test,y_pred))

# Calcular MAE
mae_modSVM_otimo = mean_absolute_error(y_test,y_pred)

In [0]:
print('mae =',mae_modSVM_otimo, 'rmse =', rmse_modSVM_otimo)

mae = 13.846142699297767 rmse = 18.566366445712546


# AIVIDADE PRÁTICA

## Considerar o conjunto de produção de cana (fa084_tch_mult.csv)

## Considerar as técnicas apresentadas hoje e nas aulas anterioes:
* KNN
* Árvore de Decisão
* Regressão Linear
* Random Forest
* GBoost (Gradient Boosting)
* XGBoost (eXtended Gradient Boosting)
* SVM

## SEMPRE QUE PERTINENTE, UTILIZAR random_state=2020

## Utilizar GridSearchCV e RandomizedSearchCV para otimizar os hiper parâmetros
* KNN - Otimizar K
* Árvore de Decisão - Otimizar max_depth, min_samples_split, min_samples_leaf
* Regressão Polinomial - Otimizar grau do polinômio
* Random Forest - Otimizar n_estimators e max_depth
* GBoost - Otimizar n_estimators e max_depth
* XGBoost - Otimizar n_estimators e max_depth
* SVM - Otimizar gamma e C

## Executar cada uma das técnicas, com os hiper parâmetros otimizados e calcular
* MAE
* RMSE

### Construir os gráficos solicitados nas aulas anteriores ('erros' e 'y_pred x y_real')

## Fazer uma discussão geral sobre os resultados obtidos com os vários modelos